In [1]:
import pandas as pd

df_train = pd.read_parquet('/Users/banubilmez/Courses/MLOps/mlops-zoomcamp-main/02-experiment-tracking/data/yellow_tripdata_2022-01.parquet')
df_val = pd.read_parquet('/Users/banubilmez/Courses/MLOps/mlops-zoomcamp-main/02-experiment-tracking/data/yellow_tripdata_2022-02.parquet')

Question 1: Number of columns in Jan 2022 Yellow Taxi Trip data

In [2]:
n_col = df_train.columns.size
n_col

19

In [3]:
def calculate_duration(df):

    df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
    df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [4]:
df_train = calculate_duration(df_train)
df_val = calculate_duration(df_val)

Question 2:  Standard deviation of the trips duration in Jan 2022 Yellow Taxi Trip data

In [5]:
df_train['duration'].std()

46.44530513776802

Question 3: Fraction of the records left after dropping the outliers

In [6]:
df_train_filtered = df_train[(df_train.duration >= 1) & (df_train.duration <= 60)]
df_val = df_val[(df_val.duration >= 1) & (df_val.duration <= 60)]

len(df_train_filtered)/len(df_train)

0.9827547930522406

Question 4: Dimensionality after OHE

In [7]:
from sklearn.feature_extraction import DictVectorizer

features = ['PULocationID', 'DOLocationID']

dv = DictVectorizer()

train_dicts = df_train_filtered[features].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[features].to_dict(orient='records')
X_val = dv.transform(val_dicts)

X_train

<2421440x515 sparse matrix of type '<class 'numpy.float64'>'
	with 4842880 stored elements in Compressed Sparse Row format>

In [8]:
target = 'duration'
y_train = df_train_filtered[target].values
y_val = df_val[target].values

Question 5: RMSE on train set

In [10]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred_train = lr.predict(X_train)
print('RMSE on train set: ', mean_squared_error(y_train, y_pred_train, squared=False))

RMSE on train set:  6.986190814952337


Question 6: RMSE on validation

In [11]:
y_pred = lr.predict(X_val)
print('RMSE on validation set: ', mean_squared_error(y_val, y_pred, squared=False))

RMSE on validation set:  7.786407163179794
